# PyTorch

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import scipy
from matplotlib import pyplot as plt

## Example 1

### Define Data

In [ ]:
a = 0.1

# rotation matrix
omega = np.pi/3;
# omega = 0
R = np.asarray([[np.cos(omega), -np.sin(omega)],[np.sin(omega), np.cos(omega)]]) 

def f_true(x):

    return scipy.special.sici((R @ x)[0] / a)[0] * np.exp(-0.5 * (np.dot(x,x)))

In [ ]:
xx = np.linspace(-2,2,100)
yy = np.linspace(-2,2,100)
plt.contourf(xx, yy, [[f_true([x_,y_]) for y_ in yy] for x_ in xx])
plt.colorbar()
plt.title("True Function")
plt.show()


In [ ]:
torch.manual_seed(123) # reproducibility

n_samples = 5000
x_train = torch.randn(n_samples,2)
y_train = torch.asarray([[f_true(x_)] for x_ in x_train.detach().numpy()],dtype=torch.float32)

x_test = torch.randn(n_samples,2) 
y_test = torch.asarray([[f_true(x_)] for x_ in x_test.detach().numpy()],dtype=torch.float32)

### Build Model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.u = nn.Sequential(nn.Linear(2,10), 
        #                        nn.Sigmoid(),
        #                        nn.Linear(10,1))
        self.u = nn.Sequential(nn.Linear(2,20), 
                               nn.ReLU(),
                                nn.Linear(20,20), 
                               nn.ReLU(),                                                            
                               nn.Linear(20,1))

    def forward(self, x):
        y = self.u(x)
        return y


In [ ]:
model = NeuralNetwork() #initialize the neural network

In [ ]:
sum(p.numel() for p in model.u.parameters())

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
model.train() # allow the model to be trained
training_losses = np.empty(0) # to store the training losses
n_epochs = 10000
for epoch in range(n_epochs):
    optimizer.zero_grad() # reset the gradients
    y_pred = model(x_train) # forward pass
    loss = loss_fn(y_pred, y_train) # compute the loss
    loss.backward() # backward pass
    optimizer.step() # update the weights
    training_losses = np.append(training_losses, loss.item() )   

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


In [ ]:
plt.plot(np.arange(n_epochs),training_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.yscale("log")
plt.title("Training Loss")
plt.show()

In [ ]:
test_loss = loss_fn( model(x_test), y_test) # compute the loss
test_loss.item()

In [ ]:
y_pred = model(x_train).detach().numpy()
y_test_pred = model(x_test).detach().numpy()

plt.scatter(y_pred, y_train,label="Training")
plt.scatter(y_test_pred, y_test,label="Testing")
plt.plot([-1,1],[-1,1],color="black",linestyle="--")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.legend()
plt.show()

In [ ]:
xx = np.linspace(-2,2,200)
yy = np.linspace(-2,2,200)
plt.contourf(xx, yy, [[model(torch.tensor([x_,y_],dtype=torch.float32)).item() for y_ in yy] for x_ in xx])
plt.colorbar()
plt.title("Regressed Function")

plt.show()


## Example 2

### Set up data

In [ ]:
batch_size_train = 100
batch_size_test = 1000

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


In [ ]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

1000 examples of 28x28

In [ ]:
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
plt.show(fig)

### Build Model

In [ ]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
MNIST_model = MNISTNet()
optimizer = optim.SGD(MNIST_model.parameters(), lr=0.01)


In [ ]:
MNIST_model.train()
n_epochs = 10
training_losses = np.empty(0) # to store the training losses

for epoch in range(n_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad() # reset the gradients
        output = MNIST_model(data) # forward pass
        loss = F.nll_loss(output, target) # compute the loss
        loss.backward() # backward pass
        optimizer.step() # update the weights
        training_losses = np.append(training_losses, loss.item() )   

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")


In [ ]:
plt.plot(np.arange(len(training_losses)),training_losses)
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.yscale("log")
plt.title("Training Loss")
plt.show()

In [ ]:
with torch.no_grad():
  output = MNIST_model(example_data)

In [ ]:
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
plt.show(fig)